In [222]:
import os
import tensorflow as tf
import random
import numpy as np
import matplotlib.pyplot as plt
import datetime
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, ZeroPadding2D, Dropout, Input
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix
import seaborn as sns
from tensorflow.keras import regularizers

In [223]:
train_data_dir = '/kaggle/input/intel-image-classification/seg_train/seg_train'
test_data_dir = '/kaggle/input/intel-image-classification/seg_test/seg_test'

In [224]:
IMG_SIZE = (150, 150)
BATCH_SIZE = 64

datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = datagen.flow_from_directory(
    train_data_dir,
    target_size= IMG_SIZE,
    batch_size= BATCH_SIZE,
    class_mode='categorical',
    shuffle=True,
    subset='training',
)

Found 11230 images belonging to 6 classes.


In [225]:
validation_generator = datagen.flow_from_directory(
    train_data_dir,
    target_size= IMG_SIZE,
    batch_size= BATCH_SIZE,
    class_mode='categorical',
    shuffle=True,
    subset='validation'
)

Found 2804 images belonging to 6 classes.


In [226]:
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size= IMG_SIZE,
    batch_size= BATCH_SIZE,
    class_mode='categorical',
    shuffle=False,
)

Found 3000 images belonging to 6 classes.


In [227]:
def display_images_per_category(generator, categories, images_per_category=3):
    # Dictionary to store images for each category
    category_images = {category: [] for category in categories}
    
    # Iterate over the dataset to fetch images
    for images, labels in generator:
        for i, label in enumerate(labels):
            category = categories[np.argmax(label)]
            if len(category_images[category]) < images_per_category:
                category_images[category].append(images[i])
        
        # Stop when we have enough images for each category
        if all(len(imgs) >= images_per_category for imgs in category_images.values()):
            break

    # Plot the images
    num_categories = len(categories)
    plt.figure(figsize=(num_categories * 2, images_per_category * 2))
    
    for col_idx, category in enumerate(categories):
        for row_idx in range(images_per_category):
            # Only plot if there are enough images
            if row_idx < len(category_images[category]):
                plt.subplot(images_per_category, num_categories, row_idx * num_categories + col_idx + 1)
                plt.imshow(category_images[category][row_idx])
                plt.axis('off')
                
                # Show category name at the top of the first row
                if row_idx == 0:
                    plt.title(category)
    
    plt.tight_layout()
    plt.show()

In [ ]:
category_names = list(train_generator.class_indices.keys())
display_images_per_category(train_generator,category_names,images_per_category = 3)

In [229]:
input_shape = train_generator.image_shape
category_count = len(category_names)

In [230]:
class AlexNet(Sequential):
    def __init__(self, input_shape, nb_classes, l2 = 0.0001):
        super().__init__()
        self.add(Input(shape=input_shape)),
        self.add(Conv2D(96, kernel_size=(11,11), strides=(4,4), activation='relu', padding='valid', kernel_regularizer=regularizers.l2(l2)))
        self.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'))
        self.add(ZeroPadding2D(padding=2))
        self.add(Conv2D(256, kernel_size=(5,5), strides=(1,1), activation='relu', kernel_regularizer=regularizers.l2(0.01)))
        self.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'))
        self.add(Conv2D(384, kernel_size=(3,3), strides=(1,1), activation='relu', padding='same', kernel_regularizer=regularizers.l2(l2)))
        self.add(Conv2D(384, kernel_size=(3,3), strides=(1,1), activation='relu', padding='same', kernel_regularizer=regularizers.l2(l2)))
        self.add(Conv2D(256, kernel_size=(3,3), strides=(1,1), activation='relu', padding='same',kernel_regularizer=regularizers.l2(l2)))
        self.add(Flatten())
        self.add(Dense(4096, activation='relu', kernel_regularizer=regularizers.l2(l2)))
        self.add(Dropout(0.5))
        self.add(Dense(4096, activation='relu', kernel_regularizer=regularizers.l2(l2)))
        self.add(Dropout(0.5))
        self.add(Dense(nb_classes, activation='softmax'))
        
        lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
            initial_learning_rate = 1e-4,
            decay_steps = 10000,
            decay_rate = 0.98
        )
        
        adam = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
        self.compile(optimizer = adam, loss = 'categorical_crossentropy', metrics = ['accuracy'])
        self.summary()

In [231]:
alex_net = AlexNet(input_shape, category_count)

Model: "alex_net_15"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_75 (Conv2D)              │ (None, 35, 35, 96)     │        34,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_30 (MaxPooling2D) │ (None, 17, 17, 96)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ zero_padding2d_15               │ (None, 21, 21, 96)     │             0 │
│ (ZeroPadding2D)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_76 (Conv2D)              │ (None, 17, 17, 256)    │       614,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_31 (MaxPooling2D) │ (None, 8, 8, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_77 (Conv2D)              │ (None, 8, 8, 384)      │       885,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_78 (Conv2D)              │ (None, 8, 8, 384)      │     1,327,488 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_79 (Conv2D)              │ (None, 8, 8, 256)      │       884,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_15 (Flatten)            │ (None, 16384)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_40 (Dense)                │ (None, 4096)           │    67,112,960 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_26 (Dropout)            │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_41 (Dense)                │ (None, 4096)           │    16,781,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_27 (Dropout)            │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_42 (Dense)                │ (None, 6)              │        24,582 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 87,666,054 (334.42 MB)

 Trainable params: 87,666,054 (334.42 MB)

 Non-trainable params: 0 (0.00 B)

In [232]:
class Model:
    def __init__(self, model, batch_size):
        self.model = model
        self.batch_size = batch_size
        
    def train(self,train_data, epochs = 50):
        history = self.model.fit(
              train_data,
              steps_per_epoch= train_data.samples // self.batch_size,
              validation_data=validation_generator,
              validation_steps=validation_generator.samples // self.batch_size,
              epochs= epochs)
        
        return history;
        
    def evaluate(self, data):
        loss, acc = self.model.evaluate(data)
        print(f"acc: {acc}, loss: {loss}")
        

In [233]:
def plot_accuracy_values(history):
        plt.plot(history.history['accuracy'])
        plt.plot(history.history['val_accuracy'])
        plt.title('Model accuracy')
        plt.ylabel('Accuracy')
        plt.xlabel('Epoch')
        plt.legend(['Train', 'Validation'], loc='upper left')
        plt.show()

        # Plot training & validation loss values
        plt.plot(history.history['loss'])
        plt.plot(history.history['val_loss'])
        plt.title('Model loss')
        plt.ylabel('Loss')
        plt.xlabel('Epoch')
        plt.legend(['Train', 'Validation'], loc='upper left')
        plt.show()

In [234]:
model = Model(alex_net, BATCH_SIZE)

In [ ]:
history = model.train(train_generator)

In [ ]:
model.model.evaluate(validation_generator)

In [ ]:
plot_accuracy_values(history)

In [ ]:
test_loss, test_acc = model.model.evaluate(test_generator, steps=test_generator.samples // BATCH_SIZE)
print(f'Test accuracy: {test_acc}')
print(f'Test loss: {test_loss}')

In [ ]:
predictions = model.model.predict(test_generator, steps=test_generator.samples // BATCH_SIZE)
predicted_classes = predictions.argmax(axis=-1)

In [ ]:
predictions = model.model.predict(test_generator, steps=len(test_generator))
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_generator.classes

cm = confusion_matrix(true_classes, predicted_classes)

class_labels = list(test_generator.class_indices.keys())

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()